# WIP Development of MPChecker: 003
 - 20190315
 - MJP
 - Initial look at MPChecker-like functionality 
 - Timing analysis of "OrbitObjectsInPointing" functionality 

In [1]:
%load_ext autoreload
%matplotlib inline

# Standard imports 
import numpy as np 
import matplotlib.pyplot as plt 
import healpy as hp
import sys, os

import MPC_library as MPCL
import phys_const as PHYS
import mpchecker 
import classes as Classes
import params 
import angles as ANG 


/Users/matthewjohnpayne/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/matthewjohnpayne/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Some of the results returned below will depend on the assumed HP scale for the preprocessing
 - E.g. the HP # for the center of the pointing ...
 - At the time of evaluation the assumed parameters are ...

In [2]:
print("Healpix: Nside", params.hp_nside)
print("Healpix: Npix", params.hp_npix)
print("Healpix: Area [deg^2]", params.hp_area)
print("Healpix: Side [deg]", params.hp_side)

Healpix: Nside 256
Healpix: Npix 786432
Healpix: Area [deg^2] 0.05245585282569793
Healpix: Side [deg] 0.22903242745449373


# Set up a list of "fake" pointings for PanSTARRS (F51) 
 - These are all on the same night, and this night is the night of one of the nbody epochs, so no significant propogation is involved

In [3]:
# Overall / average quantities 
obsCode   = 'G96'
JDutc_INT = 2458360
FOV = 3.0 ## <<-- Assuming all pointings correspond to exposures with diameter = 3.0 degrees

# Generate ~100 randomly selected pointings 
Np = 100 
rInt = np.random.choice(params.hp_npix, size=Np, replace=False)
uv_ = [hp.pix2vec(params.hp_nside, pix, nest=params.hp_nested) for pix in rInt]
HP_ = [hp.vec2pix(params.hp_nside, *UV, nest=params.hp_nested) for UV  in uv_]
'''print("# HP: ", len(HP_), len(list(set(HP_))))
print("Before, HP_ = ", HP_[0])'''
print("Before, uv_ = ", uv_[:3])

# I don't want the pointings to be in the center of each HP
# - Randomly shift by an amount which keeps them within HP
RA_, Dec_ = ANG.unit2radec(uv_)
uv_randomized = []
for i, _ in enumerate(zip(RA_, Dec_ , uv_)):
    condition = True
    while condition:
        # Generating a random shit within the approx scale of a healpix
        dR = (params.hp_side/2.)*np.random.random()
        dD = (params.hp_side/2.)*np.random.random()
        sR = +1 if np.random.random() > 0.5 else -1
        sD = +1 if np.random.random() > 0.5 else -1
        # Shift the RA,Dec
        R = _[0]+sR*dR
        D = _[1]+sD*dD
        # Get the corresponding shifted UV & HP
        UV= ANG.radec2unit(R,D)
        HP= hp.vec2pix(params.hp_nside, *UV, nest=params.hp_nested)
        # Check that the HP is the same as it was initially
        # (I'm just trying to shift the center within the same HP)
        if HP == HP_[i]:
            condition = False 
            uv_randomized.append(UV) 
uv_ = np.array(uv_randomized)
print("After , uv_ = ", uv_[:3])

# Generate multiple different pointings 
PointingList = []
for i in range(Np):
    JDutc = JDutc_INT + 0.25 + 0.5*np.random.rand()
    RA , Dec  = ANG.unit2radec(uv_[i])
    PointingList.append( Classes.Pointing(obsCode, JDutc, RA , Dec, FOV) )

# Examine some of the quantities to check that they are all as expected
print("Number of pointings generated = %d " % len(PointingList) )
print("First pointing: " , PointingList[0])
print("Last  pointing: " , PointingList[-1])
tmpR = [P.RADEC[0] for P in PointingList]
tmpD = [P.RADEC[1] for P in PointingList]
print("RA : min,median,max=",np.min(tmpR), np.median(tmpR), np.max(tmpR))
print("DEC: min,median,max=",np.min(tmpD), np.median(tmpD), np.max(tmpD))

Before, uv_ =  [(-0.48834189329962785, 0.10330547261636619, 0.86651611328125), (0.5888822104106204, -0.48936078859181936, -0.6432291666666666), (0.9182521191942846, 0.07907514460632231, 0.3880208333333333)]
After , uv_ =  [[-0.4881492   0.10224802  0.86675008]
 [ 0.58787273 -0.49008092 -0.64360418]
 [ 0.91815984  0.07755175  0.38854632]]
Number of pointings generated = 100 
First pointing:  G96_2458360.656605417_array([168.16982499])_array([60.08314658])
Last  pointing:  G96_2458360.5692884587_array([357.35422621])_array([20.99789543])
RA : min,median,max= 2.3541779613689147 188.65605089545724 357.3542262073899
DEC: min,median,max= -74.31374962520978 0.6560777344730633 74.90643583834257


# Observatory Positions wrt Center of Ecliptic
 - It will be useful later on to have ensured that each Pointing knows where it was taken from (in heliocentric coords w.r.t. the ecliptic)
 - Use the functionality demonstrated in "Demonstrate_PointingObjects.ipynb"

In [4]:
%%time 
# Do the calculations 
for P in PointingList:
    P.calc_heliocentric_position_of_observatory_in_ecliptic_coords()

# Check the results 
print("Ecliptic Helio coords of Observatory at time of first pointing: " , PointingList[0].helio_ec_posn)
print("Ecliptic Helio coords of Observatory at time of last pointing: " , PointingList[-1].helio_ec_posn)

Ecliptic Helio coords of Observatory at time of first pointing:  [ 9.25897351e-01 -4.02873794e-01  5.11108082e-05]
Ecliptic Helio coords of Observatory at time of last pointing:  [ 9.25301696e-01 -4.04243974e-01  5.09016014e-05]
CPU times: user 1.22 s, sys: 15 ms, total: 1.24 s
Wall time: 1.24 s


# Testing "OrbitObjectsInPointing"
 - A general function to perform the above methods of finding all objects within a FoV
 - This does all of the behind-the-scene calculations:
  - It calcs the HP for the fov
  - It gets the approx list of objects (preprocessed to night center)
  - It advances the approx list to the time of the pointing 
  - It selects the objects that are within the FoV

### First call to a pointing for the night is *SLOW*

In [5]:
%%time
OrbitObjectList = mpchecker.OrbitObjectsInPointing( PointingList[0] , '500' )
print(len(OrbitObjectList))

Opening '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458360_5.npy'
Opening '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/orbitObject_states_2458360_5.npy'
states[0] [ 2.95912208e-04  2.45836050e+06  1.11215415e+00 -1.77863726e+00
  1.73906687e+00 -1.14415952e-03 -8.20921128e-03 -6.44663356e-03]
Opening '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/orbitObject_desigs_2458360_5.npy'
desigs[0] 00001
86
CPU times: user 4.09 s, sys: 268 ms, total: 4.36 s
Wall time: 4.36 s


### Subsequent calls for pointings on the same night are *FAST*

In [10]:
%%time
lens_=[]
for P in PointingList[:100]:
    OrbitObjectList = mpchecker.OrbitObjectsInPointing( P , '500' )
    lens_.append(len(OrbitObjectList))
print(lens_)

[86, 45, 52, 261, 59, 50, 68, 26, 109, 86, 70, 164, 128, 49, 166, 46, 103, 49, 44, 44, 48, 80, 42, 108, 61, 78, 45, 89, 169, 34, 54, 83, 49, 32, 64, 164, 38, 285, 37, 240, 100, 79, 53, 43, 56, 35, 112, 77, 60, 137, 115, 54, 59, 52, 58, 59, 59, 52, 38, 55, 188, 53, 61, 69, 45, 53, 80, 83, 78, 121, 33, 138, 72, 46, 91, 63, 50, 218, 83, 38, 48, 62, 128, 77, 52, 52, 109, 45, 107, 38, 50, 35, 127, 59, 52, 146, 45, 102, 73, 54]
CPU times: user 414 ms, sys: 4.23 ms, total: 418 ms
Wall time: 415 ms
